date: 2024.10.30  
author: weiy  
description: 处理医疗问答数据

In [ ]:
# 将原始数据处理成问答对格式(med_data_multiQA.json)
# 数据格式是：单轮或者多轮对话的问答对格式，举例：
# [
#     {
#         "conversition": [
#             {
#                 "question": "癫痫，患者26岁，从12岁发病，发病时瞪眼，右侧身体发抖右嘴角流水，口水自行咽下，时间短。在一些小医院药物治疗过。",
#                 "answer": "您可以携带之前的检查资料，带患者前来就诊。"
#             }
#         ]
#     },
#     {
#         "conversition": [
#             {
#                 "question": "是上睑下垂还是重症肌无力，一出生右眼睛没睁开，后来睁开后就睁不大，再后来睁大一些了就有时大有时小，最大的时候几乎两个一样大，最小的时候还是遮盖半个眼球，睁大睁小和时间没关系，和吃饭说话也没关系，好像也没有出汗什么的症状，就是情绪好的时候两个眼睛大小差不多。",
#                 "answer": "应该是上睑下垂。"
#             },
#             {
#                 "question": "上睑下垂中有这种时大时小的情况吗？会不会有神经方面的问题啊。好像没用力，有时候就是突然就上去了，像最后一张照片那样，有时候几乎可以睁全，不过仰视的时候，经常会遮盖比较多，不高兴或者不舒服的时候也总是耷拉着右眼的眼皮。是不是肌无力之类的病啊，有没有其他通过吃药或者打针什么的方法可以治愈啊。",
#                 "answer": "不是肌无力，肌无力不会只影响一侧肌肉的。最后一张照片右眉高于左眉，不知是拍照角度有关还是用力造成的。"
#             },
#             {
#                 "question": "您看看这张呢，应该没有用力吧。",
#                 "answer": "是，不过右眉的形状与左侧好像不太一样，右眉可以动吗？"
#             }
#         ]
#     }
# ]

import json

with open(r'./data/train_data.json', encoding='utf-8') as f:
    datas = json.load(f)
med_datas = []
for data in datas:
    data_dict = {}
    if len(data) == 2:
        med_data = {}
        med_data['question'] = data[0].split('病人：')[1]
        med_data['answer'] = data[1].split('医生：')[1]
        data_dict['conversition'] = [med_data]
        med_datas.append(data_dict)
        # print(med_datas)
    else:
        data_dict = {}
        multi_data = []
        for i in range(0, len(data), 2):
            multi_d = {}
            multi_d['question'] = data[i].split('病人：')[1]
            multi_d['answer'] = data[i+1].split('医生：')[1]
            multi_data.append(multi_d)
        data_dict['conversition'] = multi_data
        med_datas.append(data_dict)
        # print(med_datas)
        
with open(r'./data/med_data_multiQA.json', 'w', encoding='utf-8') as f:
    json.dump(med_datas, f, ensure_ascii=False, indent=4)
        

In [3]:
# 将问答对数据转换为xtuner指令微调格式(med_data_multi_round_QA_lora.json)
# 将轮或者多轮对话的问答对格式处理为xtuner微调格式，举例：
# [
#     {
#         "conversition": [
#             {
#                 "system": "你是一个专业的医疗AI问答助手，你精通各种医疗疾病的问题，你总是以专业、礼貌的语音回答病人的问题",
#                 "input": "癫痫，患者26岁，从12岁发病，发病时瞪眼，右侧身体发抖右嘴角流水，口水自行咽下，时间短。在一些小医院药物治疗过。",
#                 "output": "您可以携带之前的检查资料，带患者前来就诊。"
#             }
#         ]
#     },
#     {
#         "conversition": [
#             {
#                 "system": "你是一个专业的医疗AI问答助手，你精通各种医疗疾病的问题，你总是以专业、礼貌的语音回答病人的问题",
#                 "input": "是上睑下垂还是重症肌无力，一出生右眼睛没睁开，后来睁开后就睁不大，再后来睁大一些了就有时大有时小，最大的时候几乎两个一样大，最小的时候还是遮盖半个眼球，睁大睁小和时间没关系，和吃饭说话也没关系，好像也没有出汗什么的症状，就是情绪好的时候两个眼睛大小差不多。",
#                 "output": "应该是上睑下垂。"
#             },
#             {
#                 "input": "上睑下垂中有这种时大时小的情况吗？会不会有神经方面的问题啊。好像没用力，有时候就是突然就上去了，像最后一张照片那样，有时候几乎可以睁全，不过仰视的时候，经常会遮盖比较多，不高兴或者不舒服的时候也总是耷拉着右眼的眼皮。是不是肌无力之类的病啊，有没有其他通过吃药或者打针什么的方法可以治愈啊。",
#                 "output": "不是肌无力，肌无力不会只影响一侧肌肉的。最后一张照片右眉高于左眉，不知是拍照角度有关还是用力造成的。"
#             },
#             {
#                 "input": "您看看这张呢，应该没有用力吧。",
#                 "output": "是，不过右眉的形状与左侧好像不太一样，右眉可以动吗？"
#             }
#         ]
#     }
# ]

import json  
  
# 读取json数据  
with open(r'./data/med_data_multiQA.json', 'r', encoding='utf-8') as file:  
    data = json.load(file)  

for conversation_dict in data:  
    conversitions = conversation_dict['conversition']  
    for conversition in conversitions:  
        # 对每一个conversition列表中的第一个字典添加system字段(系统指令)
        if conversition == conversitions[0]:  
            conversition['system'] = "你是一个专业的医疗AI问答助手，你精通各种医疗疾病的问题，你总是以专业、礼貌的语言回答病人的问题"  # 系统指令
        # 将"question"替换为"input"，"answer"替换为"output"  
        conversition['input'] = conversition.pop('question', None)  
        conversition['output'] = conversition.pop('answer', None)  
  
# 保存json数据 
with open(r'./data/med_data_multi_round_QA_lora.json', 'w', encoding='utf-8') as file:  
    json.dump(data, file, ensure_ascii=False, indent=4)  
  
print("数据处理完成！")
        

数据处理完成！
